In [2]:
!pip install pandas numpy folium matplotlib

import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from folium.plugins import HeatMap

cities = pd.DataFrame({
    "city": ["Delhi", "Mumbai", "Kolkata", "Chennai", "Hyderabad", "Bengaluru"],
    "lat": [28.7041, 19.0760, 22.5726, 13.0827, 17.3850, 12.9716],
    "lon": [77.1025, 72.8777, 88.3639, 80.2707, 78.4867, 77.5946],
})

def generate_forecast():
    days = pd.date_range(start="2025-11-01", periods=7)
    forecast_list = []
    for _, row in cities.iterrows():
        for d in days:
            base_aqi = np.random.randint(160, 300)
            base_temp = np.random.randint(28, 40)
            base_noise = np.random.randint(60, 95)
            base_industry = np.random.randint(50, 120)
            forecast_list.append([
                row["city"],
                row["lat"],
                row["lon"],
                d,
                base_aqi + np.random.randint(-20, 20),
                base_temp + np.random.randint(-3, 3),
                base_noise + np.random.randint(-5, 5),
                base_industry + np.random.randint(-10, 10)
            ])
    df = pd.DataFrame(forecast_list, columns=[
        "city", "lat", "lon", "date",
        "AQI", "temperature", "noise_level", "industrial_activity"
    ])
    return df

forecast = generate_forecast()
forecast.to_csv("7_day_hazard_forecast.csv", index=False)

def compute_hazard_score(row):
    score = (
        row["AQI"] * 0.40 +
        row["temperature"] * 0.20 +
        row["noise_level"] * 0.20 +
        row["industrial_activity"] * 0.20
    )
    return round(score, 2)

forecast["hazard_score"] = forecast.apply(compute_hazard_score, axis=1)
forecast.to_csv("forecast_with_scores.csv", index=False)

latest_day = forecast[forecast["date"] == forecast["date"].max()]

heatmap_data = []
for _, row in latest_day.iterrows():
    heatmap_data.append([row["lat"], row["lon"], row["hazard_score"]])

m = folium.Map(location=[22.0, 79.0], zoom_start=5)
HeatMap(heatmap_data, radius=35).add_to(m)
m.save("forecast_heatmap.html")

with open("forecast_report.txt", "w") as f:
    f.write("7-Day Urban Hazard Forecast Report\n")
    f.write("-----------------------------------\n\n")
    for city in cities["city"]:
        avg = forecast[forecast["city"] == city]["hazard_score"].mean()
        f.write(f"{city}: Average Hazard Score = {round(avg,2)}\n")

print("Saved: 7_day_hazard_forecast.csv")
print("Saved: forecast_with_scores.csv")
print("Saved: forecast_heatmap.html")
print("Saved: forecast_report.txt")

Saved: 7_day_hazard_forecast.csv
Saved: forecast_with_scores.csv
Saved: forecast_heatmap.html
Saved: forecast_report.txt


In [3]:
!zip -r hazard_engine_outputs.zip 7_day_hazard_forecast.csv forecast_with_scores.csv forecast_heatmap.html forecast_report.txt

  adding: 7_day_hazard_forecast.csv (deflated 72%)
  adding: forecast_with_scores.csv (deflated 70%)
  adding: forecast_heatmap.html (deflated 67%)
  adding: forecast_report.txt (deflated 55%)
